## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# Adjusting the number of shuffle partitions to match your cluster's cores and workload
spark.conf.set("spark.sql.shuffle.partitions", "4")  # Adjust the number to match your cluster's cores/workload

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
df = get_source_listing_df()
display(df.head())

In [0]:
df.shape

In [0]:
show_s3_file_contents("voc_volume/0.json")

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
from pyspark.sql.types import *

# Defining the schema of source files
bronze_schema = StructType(
    [
        StructField("date", StringType()), 
        StructField("user", StringType()),
        StructField("text", StringType()),
        StructField("sentiment", StringType()),
        StructField("source_file", StringType(), True),
        StructField("processing_time", TimestampType(), True)
    ]
)
display(bronze_schema)

In [0]:
SAMPLE_TWEETS_PATH = USER_DIR + "sample_size/"

In [0]:
SAMPLE_TWEETS_PATH = USER_DIR + "sample_size/"
files_to_copy_limit = 100

# Delete all files in the directory
dbutils.fs.rm(SAMPLE_TWEETS_PATH, recurse=True)
dbutils.fs.mkdirs(SAMPLE_TWEETS_PATH)

# Copy the first 100 files from the source directory to the destination directory
source_files = dbutils.fs.ls(TWEET_SOURCE_PATH)
files_to_copy = source_files[:files_to_copy_limit]

for file in files_to_copy:
    source_path = file.path
    destination_path = SAMPLE_TWEETS_PATH + file.name
    dbutils.fs.cp(source_path, destination_path)

In [0]:
# Checking if all files were copied
file_count = len(dbutils.fs.ls(SAMPLE_TWEETS_PATH))
print(file_count)

In [0]:
# dbutils.fs.rm(BRONZE_CHECKPOINT, True)
# dbutils.fs.rm(BRONZE_DELTA, True)

def create_directories() -> None:
    """Create the checkpoints and delta table directories"""
    dirs = [GOLD_DELTA, GOLD_CHECKPOINT, SILVER_CHECKPOINT, SILVER_DELTA, BRONZE_DELTA, BRONZE_CHECKPOINT]
    for dir in dirs:
        dbutils.fs.mkdirs(dir)
        print(f"Created \t{dir}")

create_directories()

In [0]:
def is_dir_empty():
    """Count the number of files to check if the directory is empty"""
    dirs = [GOLD_DELTA, GOLD_CHECKPOINT, SILVER_CHECKPOINT, SILVER_DELTA, BRONZE_DELTA, BRONZE_CHECKPOINT]
    for dir in dirs:
        print(f"No. of files in {dir}\t{len(dbutils.fs.ls(dir))}")

is_dir_empty()

In [0]:
from pyspark.sql.functions import input_file_name, current_timestamp
# Start a stream from SAMPLE_TWEETS_PATH with trigger
bronze_stream = (spark.readStream.format("cloudFiles")
.option("cloudFiles.format", "json")
.option("cloudFiles.schemaLocation", BRONZE_CHECKPOINT)
# .load(SAMPLE_TWEETS_PATH)
.load(TWEET_SOURCE_PATH)  # IN PRODUCTION
.withColumn("source_file", input_file_name())
.withColumn("processing_time", current_timestamp())
.writeStream
.queryName("bronze_stream")
.option("mergeSchema", "true")
.option("checkpointLocation", BRONZE_CHECKPOINT)
.trigger(processingTime='10 seconds')
.start(BRONZE_DELTA)
)

wait_stream_start(spark=spark, namedStream="bronze_stream")

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
delta_df = spark.read.format("delta").load(BRONZE_DELTA)
delta_df.createOrReplaceTempView("bronze_temp_view")

#### 4.1 Number of Tweets

In [0]:
# # Creating or replacing a temporary view to use SQL operations on stream data

bronze_stream_df = (spark.readStream
    .format("delta")
    .load(BRONZE_DELTA))

bronze_stream_df.createOrReplaceTempView("temp_bronze_table_view")

In [0]:
%sql
SELECT COUNT(*) FROM temp_bronze_table_view;

In [0]:
%sql
-- How many tweets are captured in your Bronze Table?
SELECT COUNT(*) FROM bronze_temp_view;

In [0]:
# Add a column processing time and file path
delta_df.printSchema()

In [0]:
display(delta_df.limit(5))

In [0]:
delta_df.head(5)

In [0]:
# # Printing few files from the SAMPLE to validate that the same data is present in delta table
# for file in dbutils.fs.ls(SAMPLE_TWEETS_PATH)[:5]:
#     # show_s3_file_contents(file.path.split(":")[1])
#     print(file.path)

In [0]:
# # Add a column named source file to the delta table
# show_s3_file_contents("dbfs:/tmp/labuser104917-3007340/sample_size/0.json")
# # print(SAMPLE_TWEETS_PATH)

In [0]:
delta_df.columns

#### 4.2 Are there any columns that contain Nan or Null values? If so how many and what will you do in your silver transforms to address this?

##### There doesn't seem to be null values as of now in the columns except for the "_rescued_data" column that is created by the Autoloader

In [0]:
from pyspark.sql.functions import col

# Get the list of column names
columns = delta_df.columns

# Check for null values in each column
for column in columns:
    if column != "_rescued_data":
        # Count null values in the column
        null_count = delta_df.filter(col(column).isNull()).count()
        # Print the results
        if null_count > 0:
            print(f"The column '{column}' has {null_count} null values.")
        else:
            print(f"The column '{column}' has 0 null values.")

#### 4.3 Count the number of tweets by each unique user handle and sort the data by descending count.

In [0]:
num_of_tweets_df = delta_df.groupBy("user").count().orderBy(col("count").desc())
display(num_of_tweets_df.limit(10))

#### 4.4 How many tweets have at least one mention (@) how many tweet have no mentions (@)


In [0]:
no_mention_tweets_df = delta_df.withColumn("has_atleast_one_mention", col("text").contains("@")).groupBy("has_atleast_one_mention").count()

display(no_mention_tweets_df.limit(10))

#### 4.5 Plot a bar chart that shows the top 20 tweeters (users)

In [0]:
# Display the dataframe after applying the count function on the 'user' column
tweets_count_df = delta_df.groupBy("user").count().orderBy(col("count").desc()).limit(20)

import matplotlib.pyplot as plt

# Creating a bar chart
plt.figure(figsize=(10, 6))
plt.bar(tweets_count_df.select("user").rdd.flatMap(lambda x: x).collect(), tweets_count_df.select("count").rdd.flatMap(lambda x: x).collect())
plt.xlabel("User")
plt.ylabel("Count")
plt.title("Top 20 Users by Tweet Count")

# Rotate x-axis labels by 45 degrees
plt.xticks(rotation=45)

plt.show()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

#### 5.1 setup a read stream on your bronze delta table

In [0]:
raw_bronze_stream = (spark
    .readStream
    .format("delta")
    .load(BRONZE_DELTA)
    )

In [0]:
display(raw_bronze_stream)

In [0]:
# dbutils.fs.rm(SILVER_CHECKPOINT, True)
# dbutils.fs.rm(SILVER_DELTA, True)

dbutils.fs.mkdirs(SILVER_CHECKPOINT)
dbutils.fs.mkdirs(SILVER_DELTA)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import to_timestamp, explode, udf, col, regexp_replace

# Set the configuration on the SparkSession's conf
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
format_str = "EEE MMM dd HH:mm:ss z yyyy"
pattern = r"(?<=@)\w+(?=\s)"
extract_mentions = udf(lambda text: [word[1:] for word in text.split() if word.startswith('@')], ArrayType(StringType()))

transformed_stream = (raw_bronze_stream
                      .filter(F.col("text").like("%@%"))
                      .withColumn("timestamp", to_timestamp(F.col("date"), format_str))
                      .withColumn("mention_array", extract_mentions(col("text")))
                      .select("*", explode(col("mention_array")).alias("mention"))
                      .withColumn("cleaned_text", regexp_replace(F.col("text"), r"@(\w+)\s?", ""))
                      .drop('date', 'user', 'text', 'source_file', 'processing_time', 'mention_array')
                     )

In [0]:
silver_stream = (transformed_stream
                .writeStream
                .queryName("silver_stream")
                .option("mergeSchema", "true")
                .option("checkpointLocation", SILVER_CHECKPOINT)
                .trigger(processingTime='10 seconds')
                .start(SILVER_DELTA)
)

wait_stream_start(spark=spark, namedStream="silver_stream")

In [0]:
silver_delta_df = spark.read.format("delta").load(SILVER_DELTA)
silver_delta_df.createOrReplaceTempView("silver_temp_view")

In [0]:
display(silver_delta_df)

In [0]:
%sql
SELECT COUNT(*) FROM silver_temp_view;

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# dbutils.fs.rm(GOLD_CHECKPOINT, True)
# dbutils.fs.rm(GOLD_DELTA, True)

dbutils.fs.mkdirs(GOLD_CHECKPOINT)
dbutils.fs.mkdirs(GOLD_DELTA)

In [0]:
raw_silver_stream = (spark
    .readStream
    .format("delta")
    .load(SILVER_DELTA)
)

In [0]:
import mlflow 

sentiment_analysis_udf = mlflow.pyfunc.spark_udf(spark, "models:/HF_TWEET_SENTIMENT/Production")

In [0]:
raw_gold_stream = (raw_silver_stream
               .withColumn("sentiment_id", 
                           F.when(F.col("sentiment") == "positive", 1)
                           .when(F.col("sentiment") == "negative", 0)
                           .when(F.col("sentiment") == "neutral", 2)
                           .otherwise(3))
               .withColumn("predictions", sentiment_analysis_udf(F.col("cleaned_text")))
               .withColumn("predicted_score", F.col("predictions")['score'])
               .withColumn("predicted_sentiment",
                           F.when(F.col("predictions")["label"] == "POS", "positive")
                           .when(F.col("predictions")["label"] == "NEG", "negative")
                           .when(F.col("predictions")["label"] == "NEU", "neutral")
                           .otherwise(3))
               .withColumn("predicted_sentiment_id",
                           F.when(F.col("predicted_sentiment") == "positive", 1)
                           .when(F.col("predicted_sentiment") == "negative", 0)
                           .when(F.col("predicted_sentiment") == "neutral", 2)
                           .otherwise(3))
              )

In [0]:
gold_stream = (raw_gold_stream
    .writeStream
    .queryName("gold_stream")
    .option("mergeSchema", "true")
    .option("checkpointLocation", GOLD_CHECKPOINT)
    .trigger(processingTime='10 seconds')
    .start(GOLD_DELTA)
)

wait_stream_start(spark=spark, namedStream="gold_stream")

In [0]:
gold_delta_df = spark.read.format("delta").load(GOLD_DELTA)
gold_delta_df.createOrReplaceTempView("gold_temp_view")

In [0]:
display(gold_delta_df)

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
from pyspark.sql.functions import col, cast
# Correctly use spark.sql to execute a SQL query and retrieve the data as a DataFrame
temp_df = spark.sql("SELECT * FROM gold_temp_view")

# Now, apply the withColumn transformation correctly to modify the 'predicted_sentiment_id' column
temp_df = temp_df.withColumn("predicted_sentiment_id", col("predicted_sentiment_id").cast("double")).withColumn("sentiment_id", col("sentiment_id").cast("double"))

In [0]:
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import col
import tempfile
import os

# Assuming the following variables are already set:
# - model_name: the name of your model
# - delta_table_version: version of the Delta Table (input-silver)
model_name = "m1.1"
delta_table_version = "1.0"

# Get the current MLflow version
mlflow_version = mlflow.__version__

# Start an MLflow run
with mlflow.start_run():
    # Convert DataFrame to RDD to use MulticlassMetrics
    predictionAndLabels = temp_df.select(['predicted_sentiment_id', 'sentiment_id']).rdd

    # Instantiate metrics object
    metrics = MulticlassMetrics(predictionAndLabels)

    # Compute confusion matrix
    confusion_matrix = metrics.confusionMatrix().toArray()

    # Plot confusion matrix using seaborn
    plt.figure(figsize=(10, 7))
    sns.heatmap(confusion_matrix, annot=True, fmt='g', cmap='Blues',
                xticklabels=['0.0', '1.0', '2.0'], yticklabels=['0.0', '1.0', '2.0'])
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')

    # Save the plot to a file
    with tempfile.TemporaryDirectory() as tmpdirname:
        plot_path = os.path.join(tmpdirname, "confusion_matrix.png")
        plt.savefig(plot_path)
        plt.close()

        # Log the confusion matrix plot as an artifact
        mlflow.log_artifact(plot_path)

    # Optionally compute other metrics for each label
    labels = [0.0, 1.0] # , 2.0
    precision = {label: metrics.precision(label) for label in labels}
    recall = {label: metrics.recall(label) for label in labels}
    f1 = {label: metrics.fMeasure(label) for label in labels}

    # Log metrics
    for label in labels:
        mlflow.log_metric(f"precision_{label}", precision[label])
        mlflow.log_metric(f"recall_{label}", recall[label])
        mlflow.log_metric(f"f1_score_{label}", f1[label])

    # Log model name and MLflow version as parameters
    mlflow.log_param("model_name", model_name)
    mlflow.log_param("mlflow_version", mlflow_version)

    # Log Delta Table version as a parameter
    mlflow.log_param("delta_table_version", delta_table_version)

    # End MLflow run
    mlflow.end_run()


In [0]:
# Assuming predictionAndLabels is already defined as your dataframe's RDD
metrics = MulticlassMetrics(predictionAndLabels)

# Compute confusion matrix
confusion_matrix = metrics.confusionMatrix().toArray()

# Plot confusion matrix using seaborn
plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix, annot=True, fmt='g', cmap='Blues',
            xticklabels=['0.0', '1.0', '2.0'], yticklabels=['0.0', '1.0', '2.0'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# How many mentions are there in the gold data total?
gold_delta_df.count()

In [0]:
gold_delta_df.columns
# .groupBy("mentions").agg(col("predicted_sentiment_id").distinctCount())

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

# Initialize Spark Session (if not already initialized)
spark = SparkSession.builder.appName("Sentiment Analysis").getOrCreate()

# Assuming df is your Spark DataFrame
# Group by mention and sentiment, and count each combination
mention_sentiment_counts = gold_delta_df.groupby('mention', 'predicted_sentiment').count()

# Pivot the DataFrame to transform sentiments into separate columns
pivot_df = mention_sentiment_counts.groupBy('mention').pivot('predicted_sentiment', ['negative', 'neutral', 'positive']).sum('count')

# Replace null values with zero if any column might have missing data
pivot_df = pivot_df.na.fill(0)

# Add a total column that sums up all sentiments per mention
pivot_df = pivot_df.withColumn('total', col('negative') + col('neutral') + col('positive'))

# Sort the data by the total column in descending order
pivot_df = pivot_df.orderBy(col('total').desc())

# Collect data to plot and operations which require less computation after sorting and filtering
top_20_positive = pivot_df.orderBy(col('positive').desc()).limit(20).toPandas()
top_20_negative = pivot_df.orderBy(col('negative').desc()).limit(20).toPandas()

# Plot the bar chart for the top 20 positive mentions using matplotlib
plt.figure(figsize=(12, 6))
top_20_positive.plot(kind='bar', x='mention', y='positive', color='green', legend=None)
plt.title("Top 20 Mentions with Positive Sentiment")
plt.xlabel("Mentions")
plt.ylabel("Positive Sentiment Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot the bar chart for the top 20 negative mentions
plt.figure(figsize=(12, 6))
top_20_negative.plot(kind='bar', x='mention', y='negative', color='red', legend=None)
plt.title("Top 20 Mentions with Negative Sentiment")
plt.xlabel("Mentions")
plt.ylabel("Negative Sentiment Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
stop_all_streams()

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

In [0]:
print(f"Time elapsed in the notebook:\t{(END_TIME - START_TIME)/60} mins")

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE